<a href="https://colab.research.google.com/github/yukatharsana/Blog-App-Clone_Testing/blob/main/Algorithums/detection/Yolo/YOLO_FLOWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO Flower

## Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
basicdata={'account':'sanashajeeva@gmail.com','user':"Yuka"}


In [ ]:
! locale-gen en_US.UTF-8
! export LC_ALL=en_US.UTF-8

Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


## Intallations

In [ ]:
!pip install firebase-admin

In [ ]:
!pip install ultralytics

## Imports

In [ ]:
import firebase_admin


In [ ]:
from firebase_admin import credentials,storage


In [ ]:
from firebase_admin import firestore

In [ ]:
from ultralytics import YOLO

In [ ]:
from datetime import date,datetime

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import random

In [ ]:
import yaml

In [ ]:
import csv

In [ ]:
import json

## Config

In [132]:
epochs=30
imageSize=640
optimizer='Adam'
dropout=0.1

### Dirctory

In [133]:
work_dirctory="/content/drive/MyDrive/AyushVision/yolo8/training"
work_dirctory_root="/content/drive/MyDrive/AyushVision/yolo8"
firebase_key="/content/drive/MyDrive/AyushVision/ServiceKey.json"

In [134]:
today_work_dirctory=f"{work_dirctory}/{date.today().strftime('%d_%m_%Y')}"

In [135]:
if not os.path.exists(work_dirctory):
  os.makedirs(work_dirctory,exist_ok=True)
if not os.path.exists(today_work_dirctory):
  os.makedirs(today_work_dirctory,exist_ok=True)

In [136]:
data="/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml"

In [137]:
project=f"{today_work_dirctory}/flowers/runs"
name=f'train{epochs}'

## Firebase

In [85]:
cred = credentials.Certificate(firebase_key)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'ayush-vision-asw4gh.appspot.com'
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [138]:
db = firestore.client()

In [139]:
bucket = storage.bucket()

In [140]:
yolo_ref = db.collection("training").document('yolo')
doc = yolo_ref.get()

In [141]:
if not doc.exists:
  doc_ref= db.collection("training").document('yolo')
  doc_ref.set({'algorithum':'yolo','version':'V8','mode':'detection','url':'ultralytics'})
else:
  doc_ref= yolo_ref

### Roboflow

## Model

In [142]:
today_ref=doc_ref.collection("Flower").document(datetime.now().strftime(f"%Y%m%d%H"))

In [143]:
basicdata['imageSize']=imageSize
basicdata['project']=project
basicdata['name']=name
basicdata['training_date']=datetime.now().strftime("%Y/%m/%d %H:%M:%S")
basicdata['filename']='YOLO_FLOWER'

In [144]:
with open(data, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [145]:
basicdata['names']=yaml_content['names']

In [146]:
today_ref.set(basicdata)

update_time {
  seconds: 1724220708
  nanos: 365804000
}

In [147]:
yolo_model = YOLO('yolov8n.yaml').load('yolov8n.pt')

Transferred 355/355 items from pretrained weights


#### Training

In [148]:
results = yolo_model.train(data=data,name=name, epochs=epochs, imgsz=imageSize,project=project,optimizer=optimizer,plots=True,dropout=dropout)

Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs, name=train30, exist_ok=False, pretrained=yolov8n.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

train: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/train/labels.cache... 4620 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4620/4620 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/valid/labels.cache... 441 images, 1 backgrounds, 0 corrupt: 100%|██████████| 441/441 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30      2.29G      1.304      1.046      1.639         21        640: 100%|██████████| 289/289 [02:02<00:00,  2.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:09<00:00,  1.54it/s]

                   all        441        683      0.847      0.775      0.847      0.524



30 epochs completed in 1.150 hours.
Optimizer stripped from /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.pt...
Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:08<00:00,  1.59it/s]


                   all        441        683      0.847      0.773      0.847      0.524
        Acalyphaindica         40        136      0.801      0.413      0.639      0.291
              Hibiscus         40         50      0.877       0.88       0.94      0.606
             Holybasil         40         43      0.895      0.907      0.962      0.625
        Pedalium_murex         41         85      0.827      0.675      0.812      0.511
      Red_Pea_Eggplant         39         39      0.939      0.974       0.99      0.744
        Ringworm_plant         40         44      0.819      0.864      0.832      0.516
          Sleepy_plant         40         41       0.93      0.951      0.975      0.587
           Thorn_apple         40         49      0.917      0.959       0.97      0.699
          Vitexnegundo         40         52      0.844      0.624      0.795      0.421
       adamant_creeper         40         52      0.688      0.596      0.671      0.362
   calotropis_gigante

In [149]:
args_yaml=f'{project}/{name}/args.yaml'

In [150]:
resultcsv=f'{project}/{name}/results.csv'

In [151]:
with open(args_yaml, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [152]:
today_training_colref=today_ref.collection('Train')

In [153]:
today_training_ref=today_training_colref.document('args')

In [154]:
today_training_ref.set(yaml_content)

update_time {
  seconds: 1724224920
  nanos: 230211000
}

In [155]:
todayresultref=today_training_colref.document('results')

In [156]:
trainfiles={}
wights={}

In [157]:
for file in os.listdir(f"{project}/{name}"):
  if os.path.isfile(f"{project}/{name}/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/{name}/{file}")
    blob.make_public()
    trainfiles[os.path.splitext(file)[0]]=blob.public_url


In [158]:
for file in os.listdir(f"{project}/{name}/weights"):
  if os.path.isfile(f"{project}/{name}/weights/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/weights/{file}")
    blob.upload_from_filename(f"{project}/{name}/weights/{file}")
    blob.make_public()
    wights[os.path.splitext(file)[0]]=blob.public_url

In [159]:
today_training_files=today_training_colref.document('files')
today_training_files.set(trainfiles)

update_time {
  seconds: 1724224954
  nanos: 656674000
}

In [160]:
today_weights_files=today_training_colref.document('weights')
today_weights_files.set(wights)

update_time {
  seconds: 1724224955
  nanos: 240143000
}

#### Validation

In [161]:
model_train_path=f'{project}/{name}/weights/best.pt'

In [162]:
model = YOLO(model_train_path)  # load a custom trained model

In [163]:
metrics = model.val(save_json=True,plots=True,project=project,name=f'val{epochs}')  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/valid/labels.cache... 441 images, 1 backgrounds, 0 corrupt: 100%|██████████| 441/441 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:15<00:00,  1.76it/s]


                   all        441        683      0.846      0.773      0.846      0.523
        Acalyphaindica         40        136      0.804      0.412      0.638      0.293
              Hibiscus         40         50       0.86       0.88       0.94      0.604
             Holybasil         40         43      0.893      0.907      0.962      0.626
        Pedalium_murex         41         85      0.827      0.677      0.813      0.511
      Red_Pea_Eggplant         39         39      0.939      0.974       0.99      0.744
        Ringworm_plant         40         44      0.819      0.864       0.83      0.515
          Sleepy_plant         40         41      0.931      0.951      0.975      0.584
           Thorn_apple         40         49      0.916      0.959       0.97      0.698
          Vitexnegundo         40         52      0.844      0.624      0.795      0.421
       adamant_creeper         40         52      0.688      0.596      0.669      0.362
   calotropis_gigante

In [164]:
val_today_col=today_ref.collection('Val')

In [165]:
predication_file=f'{project}/val{epochs}/predictions.json'

In [166]:
with open(predication_file, 'r') as file:
    data_dict = json.load(file)

In [167]:
val_today_ref_files=val_today_col.document('files')

In [168]:
valfiles={}

In [169]:
for file in os.listdir(f"{project}/val{epochs}"):
  if os.path.isfile(f"{project}/val{epochs}/{file}"):
    blob = bucket.blob(f"yolo/flowers/val/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/val{epochs}/{file}")
    blob.make_public()
    valfiles[os.path.splitext(file)[0]]=blob.public_url

In [170]:
val_today_ref_files.set(valfiles)

update_time {
  seconds: 1724224991
  nanos: 31619000
}

In [171]:
val_today_ref_metrics=val_today_col.document('metricsbox')

In [172]:
val_today_ref_metrics.set({'map50':metrics.box.map50,'map75':metrics.box.map75})

update_time {
  seconds: 1724224991
  nanos: 307042000
}

In [173]:
val_today_ref_metrics.update({
    'fitness':metrics.fitness
})

update_time {
  seconds: 1724224991
  nanos: 566690000
}

In [174]:
maps=np.array(metrics.maps).tolist()
boxmaps=np.array(metrics.box.maps).tolist()
p=np.array(metrics.box.p).tolist()

In [175]:
val_today_ref_metrics.update({
    'metricsmaps':maps,
    'maps':boxmaps,
    'p':p
})

update_time {
  seconds: 1724224991
  nanos: 740660000
}

In [176]:
metrics.box.map50

0.8463071393566636

In [177]:
metrics.box.map75

0.5589811836356738

In [178]:
metrics.box.maps

array([    0.29263,     0.60393,      0.6256,     0.51107,     0.74376,     0.51471,     0.58396,     0.69754,     0.42139,     0.52331,      0.3615,     0.40034])

#### Predict

In [179]:

results=model.predict(source=f'{work_dirctory_root}/dataset/ayushvision_flowers/test/images',save=True)


image 1/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0012_jpg.rf.90b4658954b2a9b3f1305f3956af7752.jpg: 640x640 1 Holybasil, 7.4ms
image 2/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0169_jpg.rf.582654067138f7912b75dc4648133b1c.jpg: 640x640 1 Holybasil, 7.3ms
image 3/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0231_jpg.rf.ad6ab6e527e0375e058347ce9d72b4db.jpg: 640x640 1 Holybasil, 18.8ms
image 4/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0232_jpg.rf.7307ae7cb6640bf4f420042ff633db08.jpg: 640x640 1 Holybasil, 7.5ms
image 5/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0246_jpg.rf.9e1d4eaab0d82565149c123dae8a477c.jpg: 640x640 1 Holybasil, 7.4ms
image 6/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_

In [180]:
predicts=[]

In [181]:
for file in os.listdir(results[0].save_dir):
  if os.path.isfile(f"{results[0].save_dir}/{file}"):
    blob = bucket.blob(f"yolo/flower/predict/{epochs}/{file}")
    blob.upload_from_filename(f"{results[0].save_dir}/{file}")
    blob.make_public()
    predicts.append(blob.public_url)

In [182]:
today_ref.collection('Predict').add({
    'files':predicts
})

(DatetimeWithNanoseconds(2024, 8, 21, 7, 25, 32, 259635, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7c836eda3d60>)

### Export Model

In [183]:
format='onnx'
model.export(format=format,keras=True,dynamic=True,simplify=True,)

Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from '/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (6.0 MB)

ONNX: starting export with onnx 1.16.2 opset 17...
ONNX: slimming with onnxslim 0.1.32...
ONNX: export success ✅ 23.7s, saved as '/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.onnx' (11.6 MB)

Export complete (25.7s)
Results saved to /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.onnx imgsz=640 data=/content/drive/My

'/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train30/weights/best.onnx'

In [184]:
blob = bucket.blob(f"yolo/flowers/model/{epochs}/best.onnx")
blob.upload_from_filename(f"{today_work_dirctory}/flowers/runs/{name}/weights/best.onnx")
blob.make_public()

In [185]:
today_ref.collection('Model').add({
    'format':format,
    "keras":True,
    "dynamic":True,
    "simplify":True,
    "model":blob.public_url
})

(DatetimeWithNanoseconds(2024, 8, 21, 7, 26, 0, 899386, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7c838e93feb0>)